In [326]:
import pandas as pd
import numpy as np
import warnings
import seaborn as sns
import matplotlib.pyplot as plt

warnings.simplefilter(action='ignore')

In [327]:
df_athletes_activities = pd.read_csv('C:/Users/USER/Desktop/EstudosDados/Projetos/Corrida/physical_inactivity_prediction/scripts_prediction_model/final/st1_df_athletes_activities.csv', parse_dates=['activity_date'])
df_athletes_activities

,athlete_id,activity_date,total_distance (km),total_time (min),pace (min/km),speed (km/h)
0,1,2020-07-09,6.03,39.0,6.47,9.28
1,1,2020-07-12,7.57,55.0,7.27,8.26
2,1,2020-07-16,3.61,23.0,6.37,9.42
3,1,2020-07-24,5.84,41.0,7.02,8.55
4,1,2020-07-25,4.28,32.0,7.48,8.02
...,...,...,...,...,...,...
739,7,2024-06-16,10.06,45.0,4.47,13.26
740,7,2024-06-18,5.20,33.0,6.35,9.27
741,7,2024-06-21,12.10,62.0,5.12,11.71
742,7,2024-06-26,4.08,22.0,5.39,11.13


In [328]:
df_athletes_activities_new = pd.DataFrame()

for id, df_grouped in df_athletes_activities.groupby('athlete_id'):

    df_grouped.insert(1, 'is_activity', True)

    # Data da primeira e última atividade para criar um range
    start, end = df_grouped['activity_date'].iloc[0], df_grouped['activity_date'].iloc[-1]

    # Colocando todas as datas em um df
    df_dates = pd.DataFrame({'activity_date': pd.date_range(start, end, freq='d')})

    # Right join pra poder manter os dados das atividades existentes mas adicionar linhas para os novos dias
    df_grouped = df_dates.merge(df_grouped, on='activity_date', how='left')

    # Preechendo os valores nulos como Dia sem atividade
    df_grouped['is_activity'].fillna(False, inplace=True)
    
    # Preenchendo novamente a coluna id para evitar NaN das linhas sem atividade
    df_grouped['athlete_id'] = id

    # Concatenação dos valores de cada atleta por id a um Dataframe
    df_athletes_activities_new = pd.concat([df_athletes_activities_new, df_grouped])

df_athletes_activities_new.reset_index(drop=True, inplace=True)
#df_athletes_activities_new.fillna(0, inplace=True)
df_athletes_activities_new

,activity_date,athlete_id,is_activity,total_distance (km),total_time (min),pace (min/km),speed (km/h)
0,2020-07-09,1,True,6.03,39.0,6.47,9.28
1,2020-07-10,1,False,NaN,NaN,NaN,NaN
2,2020-07-11,1,False,NaN,NaN,NaN,NaN
3,2020-07-12,1,True,7.57,55.0,7.27,8.26
4,2020-07-13,1,False,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
4645,2024-06-24,7,False,NaN,NaN,NaN,NaN
4646,2024-06-25,7,False,NaN,NaN,NaN,NaN
4647,2024-06-26,7,True,4.08,22.0,5.39,11.13
4648,2024-06-27,7,False,NaN,NaN,NaN,NaN


In [330]:
''' Criação de features temporais com agregação de janela deslizante '''
window_size = 7

''' Feature dependentes da is_activity '''
df_athletes_activities_new['week_frequency'] = df_athletes_activities_new.groupby('athlete_id')['is_activity'].rolling(window_size, min_periods=1).sum().values

''' Features dependentes de total_time (min) '''
df_athletes_activities_new['week_total_time'] = df_athletes_activities_new.groupby('athlete_id')['total_time (min)'].rolling(window_size, min_periods=1).sum().values
df_athletes_activities_new['week_mean_time'] = df_athletes_activities_new.groupby('athlete_id')['total_time (min)'].rolling(window_size, min_periods=1).mean().round(2).values
df_athletes_activities_new['week_max_time'] = df_athletes_activities_new.groupby('athlete_id')['total_time (min)'].rolling(window_size, min_periods=1).max().values

''' Features dependentes de total_distance (km) '''
df_athletes_activities_new['week_total_dist'] = df_athletes_activities_new.groupby('athlete_id')['total_distance (km)'].rolling(window_size, min_periods=1).sum().values
df_athletes_activities_new['week_mean_dist'] = df_athletes_activities_new.groupby('athlete_id')['total_distance (km)'].rolling(window_size, min_periods=1).mean().round(2).values
df_athletes_activities_new['week_max_dist'] = df_athletes_activities_new.groupby('athlete_id')['total_distance (km)'].rolling(window_size, min_periods=1).max().values

''' Features dependentes de pace (min/km) '''
df_athletes_activities_new['week_best_pace'] = df_athletes_activities_new.groupby('athlete_id')['pace (min/km)'].rolling(window_size, min_periods=1).min().values
df_athletes_activities_new['week_mean_pace'] = df_athletes_activities_new.groupby('athlete_id')['pace (min/km)'].rolling(window_size, min_periods=1).mean().round(2).values

''' Features dependentes de speed (km/h) '''
df_athletes_activities_new['week_best_speed'] = df_athletes_activities_new.groupby('athlete_id')['speed (km/h)'].rolling(window_size, min_periods=1).max().values
df_athletes_activities_new['week_mean_speed'] = df_athletes_activities_new.groupby('athlete_id')['speed (km/h)'].rolling(window_size, min_periods=1).mean().round(2).values

# Desconsiderando a coluna is_activity e as linhas sem atividade pois não serão mais úteis
df_athletes_activities_new = df_athletes_activities_new.loc[df_athletes_activities_new['is_activity'] == True, ~df_athletes_activities_new.columns.isin(['is_activity'])].reset_index(drop=True)

df_athletes_activities_new

,activity_date,athlete_id,total_distance (km),total_time (min),pace (min/km),speed (km/h),week_frequency,week_total_time,week_mean_time,week_max_time,week_total_dist,week_mean_dist,week_max_dist,week_best_pace,week_mean_pace,week_best_speed,week_mean_speed
0,2020-07-09,1,6.03,39.0,6.47,9.28,1.0,39.0,39.00,39.0,6.03,6.03,6.03,6.47,6.47,9.28,9.28
1,2020-07-12,1,7.57,55.0,7.27,8.26,2.0,94.0,47.00,55.0,13.60,6.80,7.57,6.47,6.87,9.28,8.77
2,2020-07-16,1,3.61,23.0,6.37,9.42,2.0,78.0,39.00,55.0,11.18,5.59,7.57,6.37,6.82,9.42,8.84
3,2020-07-24,1,5.84,41.0,7.02,8.55,1.0,41.0,41.00,41.0,5.84,5.84,5.84,7.02,7.02,8.55,8.55
4,2020-07-25,1,4.28,32.0,7.48,8.02,2.0,73.0,36.50,41.0,10.12,5.06,5.84,7.02,7.25,8.55,8.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739,2024-06-16,7,10.06,45.0,4.47,13.26,3.0,95.0,31.67,45.0,18.83,6.28,10.06,4.47,5.41,13.26,11.27
740,2024-06-18,7,5.20,33.0,6.35,9.27,3.0,108.0,36.00,45.0,20.96,6.99,10.06,4.47,5.36,13.26,11.31
741,2024-06-21,7,12.10,62.0,5.12,11.71,3.0,140.0,46.67,62.0,27.36,9.12,12.10,4.47,5.31,13.26,11.41
742,2024-06-26,7,4.08,22.0,5.39,11.13,2.0,84.0,42.00,62.0,16.18,8.09,12.10,5.12,5.26,11.71,11.42


In [331]:
# Feature para quantidade de dias desde a última atividade
df_athletes_activities_new['days_since_last_act'] = (df_athletes_activities_new.groupby('athlete_id')['activity_date'].diff().dt.days.fillna(0)).astype(int)

# Criação do target
df_athletes_activities_new['desmotivation'] = df_athletes_activities_new.groupby('athlete_id')['days_since_last_act'].transform(lambda x: (x >= 5).shift(-1).fillna(0).astype(int))

# Criação de contagem de retorno acumulado do atleta pós inatividade
df_athletes_activities_new['return_count'] = (df_athletes_activities_new.groupby('athlete_id')['desmotivation'].cumsum().shift(1).fillna(0)).astype(int)

# Criação de % acumulada de inatividade do atleta
df_athletes_activities_new['desmotivation_rate'] = df_athletes_activities_new.groupby('athlete_id')['desmotivation'].transform(lambda x: (x.cumsum()/len(x)).round(3).shift(1).fillna(0))

In [332]:
'''PR Features'''

df_athletes_activities_new['PR_total_dist'] = df_athletes_activities_new.groupby('athlete_id')['total_distance (km)'].cummax()
df_athletes_activities_new['PR_total_time'] = df_athletes_activities_new.groupby('athlete_id')['total_time (min)'].cummax()
df_athletes_activities_new['PR_pace'] = df_athletes_activities_new.groupby('athlete_id')['pace (min/km)'].cummin()
df_athletes_activities_new['PR_speed'] = df_athletes_activities_new.groupby('athlete_id')['speed (km/h)'].cummax()
df_athletes_activities_new['PR_days_since_last_act'] = df_athletes_activities_new.groupby('athlete_id')['days_since_last_act'].cummax()
df_athletes_activities_new['PR_week_frequency'] = df_athletes_activities_new.groupby('athlete_id')['week_frequency'].cummax()
df_athletes_activities_new['PR_week_total_time'] = df_athletes_activities_new.groupby('athlete_id')['week_total_time'].cummax()
df_athletes_activities_new['PR_week_max_time'] = df_athletes_activities_new.groupby('athlete_id')['week_max_time'].cummax()
df_athletes_activities_new['PR_week_total_dist'] = df_athletes_activities_new.groupby('athlete_id')['week_total_dist'].cummax()
df_athletes_activities_new['PR_week_max_dist'] = df_athletes_activities_new.groupby('athlete_id')['week_max_dist'].cummax()
df_athletes_activities_new['PR_week_best_pace'] = df_athletes_activities_new.groupby('athlete_id')['week_best_pace'].cummin()
df_athletes_activities_new['PR_week_best_speed'] = df_athletes_activities_new.groupby('athlete_id')['week_best_speed'].cummax()

In [333]:
df_athletes_activities_new

,activity_date,athlete_id,total_distance (km),total_time (min),pace (min/km),speed (km/h),week_frequency,week_total_time,week_mean_time,week_max_time,...,PR_pace,PR_speed,PR_days_since_last_act,PR_week_frequency,PR_week_total_time,PR_week_max_time,PR_week_total_dist,PR_week_max_dist,PR_week_best_pace,PR_week_best_speed
0,2020-07-09,1,6.03,39.0,6.47,9.28,1.0,39.0,39.00,39.0,...,6.47,9.28,0,1.0,39.0,39.0,6.03,6.03,6.47,9.28
1,2020-07-12,1,7.57,55.0,7.27,8.26,2.0,94.0,47.00,55.0,...,6.47,9.28,3,2.0,94.0,55.0,13.60,7.57,6.47,9.28
2,2020-07-16,1,3.61,23.0,6.37,9.42,2.0,78.0,39.00,55.0,...,6.37,9.42,4,2.0,94.0,55.0,13.60,7.57,6.37,9.42
3,2020-07-24,1,5.84,41.0,7.02,8.55,1.0,41.0,41.00,41.0,...,6.37,9.42,8,2.0,94.0,55.0,13.60,7.57,6.37,9.42
4,2020-07-25,1,4.28,32.0,7.48,8.02,2.0,73.0,36.50,41.0,...,6.37,9.42,8,2.0,94.0,55.0,13.60,7.57,6.37,9.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739,2024-06-16,7,10.06,45.0,4.47,13.26,3.0,95.0,31.67,45.0,...,4.36,13.77,5,5.0,190.0,70.0,35.81,13.24,4.36,13.77
740,2024-06-18,7,5.20,33.0,6.35,9.27,3.0,108.0,36.00,45.0,...,4.36,13.77,5,5.0,190.0,70.0,35.81,13.24,4.36,13.77
741,2024-06-21,7,12.10,62.0,5.12,11.71,3.0,140.0,46.67,62.0,...,4.36,13.77,5,5.0,190.0,70.0,35.81,13.24,4.36,13.77
742,2024-06-26,7,4.08,22.0,5.39,11.13,2.0,84.0,42.00,62.0,...,4.36,13.77,5,5.0,190.0,70.0,35.81,13.24,4.36,13.77


In [338]:
df_athletes_activities_new['desmotivation'].value_counts()

desmotivation
0    542
1    202
Name: count, dtype: int64

In [335]:
df_athletes_activities_new.dtypes

activity_date             datetime64[ns]
athlete_id                         int64
total_distance (km)              float64
total_time (min)                 float64
pace (min/km)                    float64
speed (km/h)                     float64
week_frequency                   float64
week_total_time                  float64
week_mean_time                   float64
week_max_time                    float64
week_total_dist                  float64
week_mean_dist                   float64
week_max_dist                    float64
week_best_pace                   float64
week_mean_pace                   float64
week_best_speed                  float64
week_mean_speed                  float64
days_since_last_act                int32
desmotivation                      int32
return_count                       int32
inactivity_rate                  float64
PR_total_dist                    float64
PR_total_time                    float64
PR_pace                          float64
PR_speed        

In [336]:
df_athletes_activities_new.to_csv('C:/Users/USER/Desktop/EstudosDados/Projetos/Corrida/physical_inactivity_prediction/scripts_prediction_model/final/st2_df_athletes_activities.csv', index = False)